In [ ]:
import matplotlib.pyplot as plt
from matplotlib import rc
rc('text', usetex=True)
from mpl_toolkits.axes_grid1 import make_axes_locatable

import os
import io
import gzip
import tarfile
import warnings

import numpy as np
import pandas as pd
from tqdm import tqdm

from alerce.core import Alerce
alerce = Alerce()

from astropy.time import Time
from astropy.io import fits
import astropy.units as u
import aplpy
%matplotlib inline

from astropy.wcs import WCS
from astropy import wcs

In [ ]:
ZTF_transient_list = np.load("data/ZTF_selection.npz")['transient_selection_list'].tolist()

In [ ]:
len(ZTF_transient_list)

In [ ]:
import time

In [ ]:
ZTF_transient_list

In [ ]:
def detection(data):
    

    pbar = tqdm(total=len(batch))
    ztf_time_series_images = []
    time = []
    ra = []
    dec = []

    for ztf in batch:    

        r_band = False
        detections = alerce.query_detections(ztf, format="votable")   # to get access to detected transients 
                                                              # to get the lightcurve of the transient 'ztf'
        ra_bis = []
        dec_bis = []
        time_bis = []
        transient_time_series_images = []
        for i in range(len(detections)):
            #print(detections[i]['fid'])
            if detections[i]['fid'] == 2:                         # r-band  Filter ID (1=g; 2=r; 3=i)
                #print('il a i')
                r_band = True
            
                if detections[i]['has_stamp']:
                    hdul = alerce.get_stamps(oid = ztf, candid = detections[i]['candid'])
                    
                    if hdul != None:
                        transient_time_series_images.append(np.nan_to_num(hdul[2].data[7:55,7:55]))
                        ra_bis.append(detections[i]['ra'])
                        dec_bis.append(detections[i]['dec'])
                        time_bis.append(detections[i]['mjd'])
            
        if r_band:
            ztf_time_series_images.append(transient_time_series_images)
            ra.append(ra_bis)
            dec.append(dec_bis)
            time.append(time_bis)
        
        pbar.update(1)   
        
    return time, ztf_time_series_images, ra, dec

In [ ]:
start = time.time()

batch = ZTF_transient_list[3500:]
A = detection(batch)

end = time.time()
print("The time of execution of above program is :", end-start)

In [ ]:
print(A)
#print(np.shape(A[1][0]))

# SAVE DATA

I will save the ID, the 'time', (ra,dec) which come from detection function. 
Then from catalog function, I extract and save x, y, a, b, cflux and cpeak which can be usefull for the NN (?). 
I don not save the time serie images because they would occupy a lot of place and because it is not really convenient. 

In [ ]:
import sep

def catalog(time_series, lvl):
    L = []

    for i in range(len(time_series)):
        A = time_series[i].byteswap().newbyteorder()
        bkg = sep.Background(A)
        catalog = sep.extract(A, lvl, err=bkg.globalrms)
        
        if len(catalog)!=0:
            if len(catalog)==1:
                L.append(catalog)
            elif len(catalog)>1:
                dist = [np.sqrt((catalog[i]['x']-24)**2 + (catalog[i]['y']-24)**2) for i in range(len(catalog))]
                L.append(np.array([catalog[dist.index(min(dist))]]))
            
        else: 
            while lvl>0 and len(catalog)==0:
                lvl = lvl-1
                catalog = sep.extract(A, lvl, err=bkg.globalrms)
                if len(catalog)==1:
                    L.append(catalog)
                elif len(catalog)>1:
                    dist = [np.sqrt((catalog[i]['x']-24)**2 + (catalog[i]['y']-24)**2) for i in range(len(catalog))]
                    L.append(np.array([catalog[dist.index(min(dist))]]))

    return L

In [ ]:
C = catalog(A[1][0], 6)



f = pd.DataFrame({'ID' : [ZTF_transient_list[0] for i in range(len(A[0][0]))], 'time' : A[0][0], 
                  'ra' : A[2][0], 'dec' : A[3][0], 'x' : [C[i]['x'][0] for i in range(len(C))],
                  'y' : [C[i]['y'][0] for i in range(len(C))], 'a' : [C[i]['a'][0] for i in range(len(C))], 
                 'b' :[C[i]['b'][0] for i in range(len(C))], 'cflux' : [C[i]['cflux'][0] for i in range(len(C))],
                 'cpeak' : [C[i]['cpeak'][0] for i in range(len(C))]})

for i in range(1,len(A[0])):
    C = catalog(A[1][i], 6)
    f1 = pd.DataFrame({'ID' : [ZTF_transient_list[i] for j in range(len(A[0][i]))], 'time' : A[0][i], 
                  'ra' : A[2][i], 'dec' : A[3][i], 'x' : [C[i]['x'][0] for i in range(len(C))],
                  'y' : [C[i]['y'][0] for i in range(len(C))], 'a' : [C[i]['a'][0] for i in range(len(C))], 
                 'b' :[C[i]['b'][0] for i in range(len(C))], 'cflux' : [C[i]['cflux'][0] for i in range(len(C))],
                 'cpeak' : [C[i]['cpeak'][0] for i in range(len(C))]})
    f = pd.concat([f, f1], ignore_index = True)
    f1 = 0

In [ ]:
f

In [ ]:
ID = f.drop_duplicates(subset=['ID'])
ID

In [ ]:
f.to_csv("ZTF_3500_3971.csv", index=False)

In [ ]:
plt.plot(f['ra'], f['dec'], '.')

In [ ]:
import matplotlib as mpl
plt.figure(figsize=(10,10))
plt.rc('text', usetex=True)
plt.rc('text.latex', preamble=r'\usepackage{amssymb}')
plt.rc('font', family='serif')
plt.rc('font', size=15)
mpl.rc('xtick', labelsize=20) 
mpl.rc('ytick', labelsize=20)
mpl.rc('axes', labelsize=20)

for i in list(ID['ID']):
    F = f[f['ID']==i]
    plt.plot((F['ra']-np.mean(F['ra']))*3600, (F['dec']-np.mean(F['dec']))*3600, 'r.', alpha=0.5 )

plt.xlabel('Ra', fontsize = 25)
plt.ylabel('Dec', fontsize = 25)
plt.title('Centroid position of ZTF events (band g 200 events)', fontsize = 25)
plt.xlim([-2, 2])
plt.ylim([-2, 2])
mpl.pyplot.xticks(fontsize=14);